# Exercício 2 - Agente Reativo Simples em Ambiente Completamente Observável

Implemente um novo agente reativo simples, mas agora com a vantagem de conhecer o ambiente (completamente observável). Você deve ser capaz de colocar o agente no seu simulador do exercício anterior e comparar o desempenho seguindo a mesma metodologia do exercício anterior. Compare o desempenho dos dois agentes (o deste exercício e o do anterior) neste simulador. Para o comparativo, alguns critérios do PEAS serão alterados.

- **Medida de Desempenho**: A medida de desempenho oferece o prêmio de um ponto para cada quadrado limpo em cada período de tempo ao longo de uma duração de mil períodos de tempo. Entretanto, diferentemente do exercício anterior, agora penaliza em um ponto qualquer operação de movimento que não seja NoOp e Limpar.
- **Conhecimento a priori**: A geografia do ambiente é conhecida a priori (Figura 2.2), mas a distribuição da sujeira e a posição inicial do agente não são previamente conhecidas. O ambiente para o agente é completamente observável. A aspiração limpa o quadrado atual. As ações esquerda e direita movem o agente para a esquerda e para a direita, exceto quando isso tenta levar o agente para fora do ambiente; nesse caso, o agente permanece onde está. 
- **Ações do agente**: Esquerda, Direita, Limpar e NoOp (fazer nada).
- Percepções: O agente percebe corretamente sua posição e se esta posição contém sujeita.
- **Ambiente**: mundo do aspirador de pó com apenas duas salas. Estas que por sua vez, podem estar limpas ou não conforme o tempo passa. A sujeira pode surgir de modo espontâneo para fins de simulação. 

Agora, implemente um agente reativo simples para este ambiente. Execute o simulador de ambiente com este agente para todas as configuraçoes iniciais possíveis de sujeira e posições do agente. Registre a pontuação  de desempenho dos dois agentes para cada configuração e suas pontuações médias globais.

## Imports
Para começar vamos importar algumas funções que vamos utilizar no código.

In [2]:
import random as r

## Classe: Enviroment
A classe Enviroment vai representar o ambiente, foi criada com a intenção de funcionar para n salas. Nessa classe foram implementadas as funções da criação do próprio ambiente e da criação de sujeira ao longo do tempo.

In [3]:
class Enviroment:
    FLOOR_STATE = {"clean": 0, "dirty": 1}
    
    def __init__ (self, size: int):
        self.rooms = [0] * size
    
    # Função que inicia as salas de maneira randômica.
    def init_randomly(self, dirtyRate: int):
        for i in range(len(self.rooms)):
            if r.randint(0, 100) < dirtyRate:
                self.rooms[i] = Enviroment.FLOOR_STATE["dirty"]
            else:
                self.rooms[i] = Enviroment.FLOOR_STATE["clean"]
    
    # Função que inicia as salas de acordo com o array desejado.
    def init_rooms(self, rooms: list):
        self.rooms = rooms.copy()

    # Função que suja as salas de maneira aleatória(usada no loop para sujar as salas de acordo com o tempo).
    # dirtyRate: Variável que vai de 0 a 100 que representa a porcentagem de chance de sujar cada sala.
    def rand_dirty_rooms (self, dirtyRate: int):
        for i, room in enumerate(self.rooms):
            if(room == Enviroment.FLOOR_STATE["clean"] and r.randint(0, 100) < dirtyRate):
                self.rooms[i] = Enviroment.FLOOR_STATE["dirty"]

## Classe: Agent
A classe agent representará o agente e possui a função que determina como o mesmo irá se comportar dado o ambiente. Ele possui as ações de NoOp(ficar pararo caso bater na parede), limpar(caso a sala que está esteja suja), direita e esquerda(para andar pelas salas). Como nesse exercício nós teremos dois tipos de agente, a função action é preenchida em cada agente específico.

In [4]:
class Agent:
    ACTIONS = {"NoOp": 0, "clean": 2, "right": 1, "left": -1}
    
    def __init__ (self):
        self.position = None
        
    # Função que muda a posição inicial.
    def initial_position(self, initial_position: int):
        self.position = initial_position

    # Função que representa como o agente vai agir dado o ambiente
    def action (self, env: Enviroment):
        pass
     

### Classe: Smart_agent
A classe Smart_agent vai representar a ação do agente reativo simples em ambiente completamente observável

In [5]:
class Smart_agent(Agent):
    def __init__ (self):
        Agent.__init__(self)
    
    # Função que representa como o agente vai agir dado o ambiente
    def action (self, env: Enviroment):
        # Checar se a sala atual está suja e se estiver, limpar.
        if env.rooms[self.position] == Enviroment.FLOOR_STATE["dirty"]:
            env.rooms[self.position] = Enviroment.FLOOR_STATE["clean"]
            
            return Agent.ACTIONS["clean"]
        
        distance = 0
        # Checar se tem algum chão sujo e pegar a distância, caso não tenha sujeira a distância sera 0.
        for i in range(len(env.rooms)):
            if env.rooms[i] == Enviroment.FLOOR_STATE["dirty"]:
                if abs(i - self.position) < abs(distance) or distance == 0:
                    distance = i - self.position
        
        # Ir para direita se a distância for positiva, para esquerda se negativa e não fazer nada se distância for igual a zero.
        decision = Agent.ACTIONS["right"] if distance > 0 else Agent.ACTIONS["left"] if distance < 0 else Agent.ACTIONS["NoOp"]
        
        self.position += decision
            
        return decision

### Classe: Dumb_agent
A classe Dumb_agent vai representar a ação do agente reativo simples em ambiente parcialmente observável

In [6]:
class Dumb_agent(Agent):
    def __init__ (self):
        Agent.__init__(self)
    
    # Função que representa como o agente vai agir dado o ambiente
    def action (self, env: Enviroment):
        # Checar se a sala atual está suja e se estiver, limpar.
        if env.rooms[self.position] == Enviroment.FLOOR_STATE["dirty"]:
            env.rooms[self.position] = Enviroment.FLOOR_STATE["clean"]
            
            return Agent.ACTIONS["clean"]
        
        # Decidir entre ir para esquerda ou direita de maneira aleatória com 50% de chande para cada lado.
        decision = Agent.ACTIONS["right"] if r.randint(0, 100) < 50 else Agent.ACTIONS["left"]
        # Verificar se bateu na parede, caso bater mudar decisão
        if (decision == Agent.ACTIONS["left"] and self.position == 0) or (decision == Agent.ACTIONS["right"] and self.position == len(env.rooms) - 1):
            decision = Agent.ACTIONS["NoOp"]
        
        self.position += decision
            
        return decision

## Classe: Evaluator
A classe evaluator vai representar a avaliação do agente no ambiente, foi criada com a intenção de funcionar para n passos(quantas iterações necessárias). Nessa classe foi implementada a lógica de pontuação do agente de acordo com suas ações(Dar 1 ponto sempre que limpar uma sala e perder 1 quando andar para direita ou esquerda).

In [7]:
class Evaluator:
    def __init__ (self, env: Enviroment, agent: Agent, steps: int, seed: int, dirtyRate: int):
        self.env = env
        self.agent = agent
        self.steps = steps
        r.seed(seed)
        self.dirtyRate = dirtyRate
        self.agent_points = 0
    
    # Função que inicia a simulação
    def execute (self):
        self.agent_points = 0
        for i in range(self.steps):
            # Pontuar o agente de acordo com a sua ação.
            self.evaluate_agent(self.agent.action(self.env))
            # Sujar sala de maneira aleatória.
            self.env.rand_dirty_rooms(self.dirtyRate)
        
        return self.agent_points
    
    # Função que avalia o agente.
    def evaluate_agent (self, action: int):
        if action == Agent.ACTIONS["clean"]:
            self.agent_points += 1
        elif action == Agent.ACTIONS["right"] or action == Agent.ACTIONS["left"]:
            self.agent_points -= 1

## Rodando os testes
Utilizando as classes para criar os testes pedidos para 2 salas.

In [ ]:
# Criando as variáveis das classes.
env = Enviroment(2)
smart_agent = Smart_agent()
dumb_agent = Dumb_agent()
eval_smart = Evaluator(env, smart_agent, 1000, 123, 30)
eval_dumb = Evaluator(env, dumb_agent, 1000, 123, 30)

# Variáveis que representam os testes.
env_possibilities = [[0, 0], [0, 1], [1, 0], [1, 1]]
agent_possibilities = [0, 1]
result_smart = []

# Executando e printando os resultados.
print("Agente Reativo Simples em Ambiente Completamente Observável:")
for agent_possibility in agent_possibilities:
    print(f"\tPosição inicial {agent_possibility}: ")

    for env_possibility in env_possibilities:
        smart_agent.initial_position(agent_possibility)
        env.init_rooms(env_possibility)
        result_smart.append(eval_smart.execute())
        print(f"\t\tAmbiente inicial: {env_possibility} | Pontuação: {result_smart[-1]}")
        
    print()
print(f"\tMédia das pontuações: {sum(result_smart)/len(result_smart)}")

print()
print()

result_dumb = []
print("Agente Reativo Simples em Ambiente Parcialmente Observável:")
for agent_possibility in agent_possibilities:
    print(f"\tPosição inicial {agent_possibility}: ")
    
    for env_possibility in env_possibilities:
        dumb_agent.initial_position(agent_possibility)
        env.init_rooms(env_possibility)
        result_dumb.append(eval_dumb.execute())
        print(f"\t\tAmbiente inicial: {env_possibility} | Pontuação: {result_dumb[-1]}")
        
    print()
print(f"\tMédia das pontuações: {sum(result_dumb)/len(result_dumb)}")